#Problem Statement

## Business Context

Teachers, Parents and students from Grade 5 provided commercial solutions provided by school that adheres TEA curriculam during academic year.


*   Teachers or instructors want to challenge students with different varity of
questions to guauge their experitise level.
*   Parents with limited speaking ability to english face challenge to understand and explian concept mathematical concept to student.
* Students who want to understand concept in friendly manner and as per TEKS curricularm is limited accesibility.

To addres above challenges, There is need a system that can help above audience to leverage modern system to understand mathematical concepts.


### Objective


Create a RAG based chat agent that uses knowledge deisgn by TEKS and helps students, teacher or parents to understand concepts of mathematics from Grade 5 of Texas.


# Installing required libraries and dependencies

In [ ]:

# Installing core libraries for RAG pipeline and LLM integration
# huggingface_hub       → Load models and datasets from Hugging Face
# tiktoken              → Tokenizer used by OpenAI-compatible models
# pymupdf               → PDF loader for document parsing
# langchain             → Framework for building LLM pipelines (RAG, chains )
# langchain-community   → Community-maintained integrations (e.g., loaders, embeddings)
# chromadb              → Lightweight vector store for document retrieval
# sentence-transformers → Pretrained models for generating vector embeddings
# numpy, pandas         → Core numerical and data processing libraries

!pip install huggingface_hub tiktoken pymupdf langchain langchain-community chromadb sentence-transformers numpy pandas langchain-text-splitters openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105

Name: openai
Version: 2.15.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [ ]:
# General-purpose libraries for file I/O and data processing
import json          # For handling JSON responses and configs
import os            # For file path operations
import pandas as pd  # For working with structured tabular data
import tiktoken      # For encoding/decoding tokens used in LLMs

# LangChain ecosystem for document loading, chunking, embedding, and retrieval
from langchain_text_splitters import RecursiveCharacterTextSplitter      # Splits long text into manageable chunks
from langchain_community.document_loaders import PyMuPDFLoader          # Loads and parses PDF documents
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings  # Creates embeddings using pretrained models
from langchain_community.vectorstores import Chroma                     # Vector database for similarity search

# HuggingFace and LLaMA Inference
from huggingface_hub import hf_hub_download  # Downloads models/data from HuggingFace Hub

import openai
import os
from google.colab import userdata

SyntaxError: invalid syntax (ipython-input-609340015.py, line 1)

# Defining LLM Model

In [ ]:
# define OpenAI mode that will be used.
GPT_MODEL= "gpt-4.1-nano"

In [ ]:
#create LLM connector instance by referring openai key
client = openai.OpenAI(api_key=userdata.get("Math5"))

## Prompt Engineering

### Prompt

In [ ]:
#defining prompt
dev_prompt = """
You are a Texas Grade 5 Mathematics tutor for kids, and you also support parents and teachers.
Your tone must be kid-safe, friendly, clear, and encouraging. Keep explanations simple, accurate, and non-judgmental.
CRITICAL OUTPUT RULES:
- Output MUST be valid JSON only (no markdown, no code fences, no extra text).
- Use double quotes for all JSON keys/strings.
- Do not include trailing commas.
- Keep responses safe for kids (no unsafe, hateful, sexual, violent, or scary content).
SCOPE RULE:
- Only answer mathematics (Grade 5 level preferred; you may briefly define advanced terms if asked).
- If the user asks anything not related to mathematics, respond ONLY with:
  {"type":"Refusal","message":"Sorry, I can't answer questions other than mathematics."}
OUTPUT TYPES:
1) Concept explanation:
{
  "type": "Concept",
  "message": "Short kid-friendly explanation..."
}
2) Practice questions (MCQ):
- Create up to 5 questions maximum. If user requests more than 5, generate only 5.
- Each question MUST have exactly 4 answer choices.
- The correct answer MUST be one of the 4 choices.
- Provide the correct answer explicitly using "CorrectOption" (A/B/C/D) and "CorrectAnswer" (exact matching text from Answers).
- Keep math appropriate and compute accurately. Avoid trick questions.
{
  "type": "Questions",
  "message": [
    {
      "Q1": "Question text",
      "Answers": { "A": "Option 1", "B": "Option 2", "C": "Option 3", "D": "Option 4" },
      "CorrectOption": "B",
      "CorrectAnswer": "Option 2"
    }
  ]
}
ACCURACY / ANTI-HALLUCINATION RULES:
- Do the math carefully. Ensure only one correct option unless the user explicitly asks for multiple correct answers.
- If you detect ambiguity (missing numbers/units), ask ONE clarifying question using:
  {"type":"Concept","message":"I need one detail to answer: ..."}
  (Keep it math-only and kid-safe.)
STYLE GUIDELINES:
- Use simple words and short sentences for kids.
- For parents/teachers, add a brief note on how to support learning (1–2 sentences).
- Avoid unrelated topics, brand names, or personal data requests.
""".strip()

### Output schema

In [ ]:
#JSON schema for output

JSON_SCHEMA = {
  "type": "object",
  "additionalProperties": False,
  "required": ["type", "message"],
  "properties": {
    "type": {"type": "string", "enum": ["Concept", "Questions", "Refusal"]},
    "message": {
      "anyOf": [
        {"type": "string"},
        {
          "type": "array",
          "maxItems": 5,  # ✅ hard limit: cannot output 15
          "items": {
            "type": "object",
            "additionalProperties": False,
            "required": ["Question", "Answers", "CorrectOption", "CorrectAnswer"],
            "properties": {
              "Question": {"type": "string"},
              "Answers": {
                "type": "object",
                "additionalProperties": False,
                "required": ["A", "B", "C", "D"],
                "properties": {
                  "A": { "type": "string" },
                  "B": { "type": "string" },
                  "C": { "type": "string" },
                  "D": { "type": "string" }
                }
              },
              "CorrectOption": {"type": "string", "enum": ["A", "B", "C", "D"]},
              "CorrectAnswer": {"type": "string"}
            }
          }
        }
      ]
    }
  }
}

In [ ]:
print(dev_prompt)

### LLM function to get output

#### testing of function

In [ ]:
get_LLMResponse("summarize decimal")

{"type":"Concept","message":"A decimal is a way to write numbers that are not whole, using a dot called a decimal point. The digits after the decimal point show parts of a whole, like halves, quarters, or tenths. For example, 0.5 means half, and 0.75 means three quarters. \n\nFor parents/teachers: Help kids understand that decimals are just another way to write parts of a whole, similar to fractions. Use real-life examples like money to make it clearer."}
ResponseUsage(input_tokens=735, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=109, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=844)


In [ ]:
get_LLMResponse("give me 5 problems for algebra")

{"type":"Questions","message":[{"Question":"If x + 3 = 7, what is x?","Answers":{"A":"2","B":"4","C":"5","D":"3"},"CorrectOption":"B","CorrectAnswer":"4"},{"Question":"What is the value of y in the equation 2y = 10?","Answers":{"A":"5","B":"10","C":"20","D":"8"},"CorrectOption":"A","CorrectAnswer":"5"},{"Question":"Solve for z: z - 4 = 6.","Answers":{"A":"2","B":"10","C":"4","D":"8"},"CorrectOption":"B","CorrectAnswer":"10"},{"Question":"If 3a = 15, what is a?","Answers":{"A":"5","B":"3","C":"15","D":"6"},"CorrectOption":"A","CorrectAnswer":"5"},{"Question":"What is the value of x in the equation 5x = 25?","Answers":{"A":"10","B":"5","C":"25","D":"20"},"CorrectOption":"B","CorrectAnswer":"5"}]}
ResponseUsage(input_tokens=738, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=238, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=976)


# Data preparation for RAG

## Loading documents

In [ ]:
#mount google drive to accss manual PDF
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#file path
math_file_path= "/content/grade5-teks-062024-0.pdf"

In [ ]:
# Initialize PDF loader using LangChain's PyMuPDFLoader
# This loads the document in a format suitable for text chunking and embedding
pdf_loader = PyMuPDFLoader(math_file_path)
page_docs = pdf_loader.load()  # usually 1 Document per page

# Example: keep pages 5–10
keep_pages_0_based = set(range(5, 10))

#extract course details about mathematics only.
selected_pages = [
    d for d in page_docs
    if d.metadata.get("page") in keep_pages_0_based
]

print("Total pages:", len(page_docs))
print("Selected pages:", len(selected_pages))
print("Selected page numbers:", sorted({d.metadata["page"] for d in selected_pages}))

Total pages: 40
Selected pages: 5
Selected page numbers: [5, 6, 7, 8, 9]


In [ ]:
# Check the metadata and content of the first page/document
print(selected_pages[1].metadata)
print(selected_pages[1].page_content[:300])  # preview first 300 characters of content

{'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-07-16T11:34:41-05:00', 'source': '/content/grade5-teks-062024-0.pdf', 'file_path': '/content/grade5-teks-062024-0.pdf', 'total_pages': 40, 'format': 'PDF 1.7', 'title': 'Grade5_TEKS_062024', 'author': 'Cami Jones', 'subject': '', 'keywords': '', 'moddate': '2024-07-16T16:44:55-05:00', 'trapped': '', 'modDate': "D:20240716164455-05'00'", 'creationDate': "D:20240716113441-05'00'", 'page': 6}
revised August 2022 
7 
grade and course is intentional. The process standards weave the other knowledge and skills 
together so that students may be successful problem solvers and use mathematics efficiently and 
effectively in daily life. The process standards are integrated at every grade level a


## Data Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',  # Token encoding compatible with OpenAI models
    chunk_size=200,               # Number of tokens per chunk
    chunk_overlap=20              # Overlap to preserve context between chunks
)


In [ ]:
#create chunks for given documents
# Split the PDF content into smaller overlapping chunks for embedding
# Combines loading and splitting using the defined text_splitter
#document_chunks = pdf_loader.load_and_split(text_splitter)
document_chunks = text_splitter.split_documents( selected_pages)

In [ ]:
# Preview number of chunks
print(f"Total Chunks Created: {len(document_chunks)}")

Total Chunks Created: 22


In [ ]:
#check content of first chunk,  a sample to validate
document_chunks[3].page_content

'knowledge and skills for mathematics, guided by the college and career readiness standards. By \nembedding statistics, probability, and finance, while focusing on computational thinking, \nmathematical fluency, and solid understanding, Texas will lead the way in mathematics education \nand prepare all Texas students for the challenges they will face in the 21st century. \n(2) \nThe process standards describe ways in which students are expected to engage in the content. The \nplacement of the process standards at the beginning of the knowledge and skills listed for each'

## Embedding

In [ ]:
# using General Text Embeddings (GTE) model , https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')


# Step: Load Embedding Model
# Using HuggingFace's General Text Embeddings (GTE) model suitable for semantic search & RAG
# Reference: https://huggingface.co/thenlper/gte-large

embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')


print("\nEmbedding model loaded: thenlper/gte-large")


/tmp/ipython-input-215520886.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]


Embedding model loaded: thenlper/gte-large


In [ ]:
# Generate sample embeddings for first two chunks to validate model output
embedding_1 = embedding_model.embed_query(document_chunks[3].page_content)

In [ ]:
# print dimention of vector and compare lenght of two embeddings.
print("Dimension of the embedding vector ",len(embedding_1))

Dimension of the embedding vector  1024


## Vector Database

In [ ]:
#create a directory to store vector db
out_dir = 'math5_data'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [ ]:
# Create a Chroma vector store from the document chunks using the embedding model
# Persist the vector index to the specified directory (out_dir)

out_dir = "./chroma_db"  # Define output directory if not done earlier

vectorstore = Chroma.from_documents(
    documents=document_chunks,
    embedding=embedding_model,
    persist_directory=out_dir
)

print(f"Chroma vectorstore created and saved at: {out_dir}")

Chroma vectorstore created and saved at: ./chroma_db


In [ ]:
# Reload the Chroma vector store from disk (useful after notebook restart or deployment)

vectorstore = Chroma(
    persist_directory=out_dir,
    embedding_function=embedding_model
)

print("Vectorstore successfully loaded from disk.")

Vectorstore successfully loaded from disk.


/tmp/ipython-input-2749883970.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [ ]:
# Confirm the embedding model is correctly attached to the vectorstore
print("Embedding function used in vectorstore:")
print(vectorstore.embeddings)

Embedding function used in vectorstore:
client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='thenlper/gte-large' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


### Semantic search on vectorstore

In [ ]:
# Perform semantic search on vectorstore with a medical keyword query
# Retrieve top 3 most relevant document chunks

query = "postive rational numbers"
results = vectorstore.similarity_search(query, k=3)

# Preview results
for i, doc in enumerate(results):
    print(f"--- Result {i+1} ---")
    print(doc.page_content[:400])  # Display first 400 characters for readability

--- Result 1 ---
compare, and order positive rational numbers and understand relationships as related to place 
value. The student is expected to: 
(A) 
represent the value of the digit in decimals through the thousandths using expanded 
notation and numerals; 
(B) 
compare and order two decimals to thousandths and represent comparisons using the 
symbols >, <, or =; and 
(C) 
round decimals to tenths or hundredth
--- Result 2 ---
perseverance. Students in Grade 5 are expected to perform their work without the use of 
calculators. 
(4) 
The primary focal areas in Grade 5 are solving problems involving all four operations with 
positive rational numbers, determining and generating formulas and solutions to expressions, and 
extending measurement to area and volume. These focal areas are supported throughout the 
mathematical
--- Result 3 ---
revised August 2022 
9 
(J) 
represent division of a unit fraction by a whole number and the division of a whole 
number by a unit fraction such as

## Retriever

In [ ]:
# Initialize retriever from vectorstore using similarity search (cosine similarity)
# Retrieves top 3 most relevant chunks for any input query
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


In [ ]:
# Run retriever to find top 3 document chunks relevant to the Cataract question
query = "What is algebric reasoning?"
rel_docs = retriever.invoke(query)

# View retrieved chunks
for i, doc in enumerate(rel_docs):
    print(f"Result {i+1}:")
    print(doc.page_content[:400])  # Display first 400 characters

Result 1:
abstraction to solve problems. Students will effectively communicate mathematical ideas, 
reasoning, and their implications using multiple representations such as symbols, diagrams, 
graphs, computer programs, and language. Students will use mathematical relationships to 
generate solutions and make connections and predictions. Students will analyze mathematical 
relationships to connect and commu
Result 2:
(B) 
use a problem-solving model that incorporates analyzing given information, formulating 
a plan or strategy, determining a solution, justifying the solution, and evaluating the 
problem-solving process and the reasonableness of the solution;
Result 3:
numbers using equations with a letter standing for the unknown quantity; 
(C) 
generate a numerical pattern when given a rule in the form y = ax or y = x + a and graph; 
(D) 
recognize the difference between additive and multiplicative numerical patterns given in 
a table or graph; 
(E) 
describe the meaning of parenthese

## Function to get context from documents

In [ ]:
def generate_context_from_input(user_input):
 """
 Retrieve relevent chunks from vector store and genrate context
 """
 rel_docs = retriever.invoke(query)
 context_list = [ d.page_content   for d in rel_docs]
 context_for_query = ". ".join (context_list)

 return context_for_query


In [ ]:
print(generate_context_from_input("What is algebric reasoning?"))

abstraction to solve problems. Students will effectively communicate mathematical ideas, 
reasoning, and their implications using multiple representations such as symbols, diagrams, 
graphs, computer programs, and language. Students will use mathematical relationships to 
generate solutions and make connections and predictions. Students will analyze mathematical 
relationships to connect and communicate mathematical ideas. Students will display, explain, or 
justify mathematical ideas and arguments using precise mathematical language in written or oral 
communication. 
(3) 
For students to become fluent in mathematics, students must develop a robust sense of number. 
The National Research Council's report, "Adding It Up," defines procedural fluency as "skill in 
carrying out procedures flexibly, accurately, efficiently, and appropriately." As students develop 
procedural fluency, they must also realize that true problem solving may take time, effort, and. (B) 
use a problem-solving mod

### experimenting with RAG function

In [ ]:
def get_LLMResponse(user_input: str, context: str = ""):
  messages = [
      {
          "role": "developer",
          "content": dev_prompt
      }
  ]

  # Add context as a separate message (optional)
  if context and context.strip():
      messages.append({
          "role": "developer",
          "content": (
              "Use the following CONTEXT only if it is relevant to the user's request. "
              "Do not invent facts that are not in the context.\n"
              "BEGIN_CONTEXT\n"
              f"{context}\n"
              "END_CONTEXT"
          )
      })

  # User request last
  messages.append({
      "role": "user",
      "content": user_input
  })

  response = client.responses.create(
      model=GPT_MODEL,
      input=messages,
      temperature=0.2,
      max_output_tokens=800,
      text={
          "format": {
              "type": "json_schema",
              "name": "grade5_math_response",
              "strict": True,
              "schema": JSON_SCHEMA
          }
      }
  )
  return response.output_text
  #print(response.output_text)
  #print(response.usage)


In [ ]:
def generate_response_from_rag(user_input:str =""):
  context = generate_context_from_input(user_input)

  llm_response = get_LLMResponse(user_input=user_input, context= context)
  return llm_response



In [ ]:
response = generate_response_from_rag ("give me 5 problems for decimals ")
print(response)


{"type":"Questions","message":[{"Question":"What is 0.75 + 0.25?","Answers":{"A":"1.00","B":"0.50","C":"1.25","D":"0.75"},"CorrectOption":"A","CorrectAnswer":"1.00"},{"Question":"Subtract 0.6 from 1.2.","Answers":{"A":"0.6","B":"1.8","C":"0.12","D":"0.66"},"CorrectOption":"A","CorrectAnswer":"0.6"},{"Question":"Multiply 0.3 by 0.4.","Answers":{"A":"0.12","B":"0.7","C":"0.03","D":"0.7"},"CorrectOption":"A","CorrectAnswer":"0.12"},{"Question":"Divide 0.9 by 3.","Answers":{"A":"0.3","B":"0.03","C":"0.09","D":"3.0"},"CorrectOption":"A","CorrectAnswer":"0.3"},{"Question":"Which decimal is the largest?","Answers":{"A":"0.45","B":"0.5","C":"0.55","D":"0.5"},"CorrectOption":"C","CorrectAnswer":"0.55"}]}


In [ ]:
response = generate_response_from_rag ("explain algebra in friendly manner ")
print(response)

{"type":"Concept","message":"Algebra is like a puzzle where you use symbols, usually letters, to stand for numbers we don't know yet. We use these symbols to write equations and find out what the missing numbers are. For example, if you see x + 3 = 5, you can figure out that x is 2 because 2 + 3 = 5. \n\nFor parents/teachers: Algebra helps kids develop problem-solving skills and understand how different parts of math connect. Encourage children to think of symbols as placeholders for numbers and practice solving simple equations to build confidence."}


## Setting up a Hugging Face Docker Space for the Backend


In [ ]:
!pip install --upgrade --force-reinstall huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/11

In [ ]:
from huggingface_hub import login, HfApi

In [ ]:
from google.colab import userdata


In [ ]:
BACKEND_REPO_NAME="Grade5MathBackend"
hf_user_id="TokenTutor"
repo_id = hf_user_id + "/" + BACKEND_REPO_NAME


hf_token = userdata.get('hf_token')


In [ ]:
# Import the login function from the huggingface_hub library
from huggingface_hub import login

# Login to your Hugging Face account using your access token
login(token=hf_token)

# Import the create_repo function from the huggingface_hub library
from huggingface_hub import create_repo

In [ ]:
# Try to create the repository for the Hugging Face Space
try:
    create_repo(BACKEND_REPO_NAME,  # One can replace "Backend_Docker_space" with the desired space name
        repo_type="space",  # Specify the repository type as "space"
        space_sdk="docker",  # Specify the space SDK as "docker" to create a Docker space
        private=False  # Set to True if you want the space to be private
    )
except Exception as e:
    # Handle potential errors during repository creation
    if "RepositoryAlreadyExistsError" in str(e):
        print("Repository already exists. Skipping creation.")
    else:
        print(f"Error creating repository: {e}")

Error creating repository: Client error '409 Conflict' for url 'https://huggingface.co/api/repos/create' (Request ID: Root=1-6976b36d-3e788dc06e9c19346b85af4e;2091219a-f126-4b95-8649-293223204772)
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409

You already created this space repo: TokenTutor/Grade5MathBackend


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Uploading Files to Hugging Face Space (Docker Space)

In [ ]:
login(token=hf_token)

# Initialize the API
api = HfApi()

# Upload Streamlit app files stored in the folder called deployment_files
api.upload_folder(
    folder_path="/content/backend_files",  # Local folder path in azureml
    repo_id=repo_id,  # Hugging face space id
    repo_type="space"  # Hugging face repo type "space"
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...6351d9b6a/data_level0.bin:  93%|#########2|  393kB /  424kB            

  ...4-f776351d9b6a/header.bin: 100%|##########|   100B /   100B            

  ...4-f776351d9b6a/length.bin: 100%|##########|   400B /   400B            

  .../chroma_db/chroma.sqlite3: 100%|##########|  524kB /  524kB            

CommitInfo(commit_url='https://huggingface.co/spaces/TokenTutor/Grade5MathBackend/commit/32abab0d46943aeae607de93f60eeef0f5d69f5c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='32abab0d46943aeae607de93f60eeef0f5d69f5c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/TokenTutor/Grade5MathBackend', endpoint='https://huggingface.co', repo_type='space', repo_id='TokenTutor/Grade5MathBackend'), pr_revision=None, pr_num=None)

# Inferencing using Flask API

In [ ]:
import json  # To handle JSON formatting for API requests and responses
import requests  # To send HTTP requests to the deployed Flask API

import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations

In [ ]:
hf_user_id="TokenTutor"
model_root_url = "https://" + hf_user_id + "-" + BACKEND_REPO_NAME + ".hf.space"  # Base URL of the deployed Flask API on Hugging Face Space; enter user name and space name before running the cell

print(model_root_url)

https://TokenTutor-Grade5MathBackend.hf.space


In [ ]:
model_url = model_root_url + "/MathQuestion"  # Endpoint for online (single) inference
print(model_url)

https://TokenTutor-Grade5MathBackend.hf.space/MathQuestion


In [ ]:
payload = {
        "Query": "give me info about us news "
    }

In [ ]:
response = requests.post(model_url, json=payload)

response

<Response [200]>

In [ ]:
json_data= response.json()
#response.get('Prediction')
print( 'result ', json_data )

result  {'message': "Sorry, I can't answer questions other than mathematics.", 'type': 'Refusal'}
